# This notebook comparing the GAFF1 modelling data with the experimental value, taking the input from clustering.ipynb, in which all the results are in sampling/

In [1]:
import mdtraj as md
import numpy as np

In [7]:
def compute_ensemble_avg_NOE(grouped_files, indices, verbose=False):

    #indices = np.loadtxt(indices, dtype=int)
    results = []
    for i,state in enumerate(grouped_files):
        distances = md.compute_distances(md.load(state), indices)*10. # convert nm to Å
        #print(distances.shape)
        data = np.mean(np.array(distances), axis=0)
        results.append(data)
        #print(np.mean([d[0][1] for d in distances]))
        #print(data)
        #exit()
        #np.savetxt(f'{outdir}/NOE_{i}.txt', data)
    return results

In [10]:
import numpy as np

# Sample gmx_indices and restraint_indices
gmx_indices = [
    [12, 31],
    [12, 32],
    [13, 31],
    [13, 32],
    [40, 53],
    [40, 54],
    [41, 53],
    [41, 54],
]

d_indices = np.array(gmx_indices) - 1  # Convert to 0-based indexing
restraint_indices = np.array([0, 0, 0, 0, 1, 1, 1, 1])  # Provided restraint_indices
    

for cluster in range(6): 
    # Assuming pdb is a list of pdb file paths
    pdb = [f"sampling/sample_state_0_cluster{cluster}_whole.pdb"]

    # Compute distances
    distances = compute_ensemble_avg_NOE(pdb, d_indices)
    distances = np.array(distances).flatten()

    # Initialize dictionary to hold distances for each state
    state_distances = {state: [] for state in np.unique(restraint_indices)}

    # Assign distances to the corresponding state
    for state in state_distances:
        state_distances[state] = distances[restraint_indices == state]
    
    # Calculate average distance for each state
    avg_state_distances = {state: np.mean(dists) for state, dists in state_distances.items()}
    print(f"For Cluster {cluster}, the average state distances are: {avg_state_distances}")


For Cluster 0, the average state distances are: {0: 3.1540895, 1: 4.649834}
For Cluster 1, the average state distances are: {0: 3.1926737, 1: 4.654296}
For Cluster 2, the average state distances are: {0: 3.1588025, 1: 3.189073}
For Cluster 3, the average state distances are: {0: 3.1444707, 1: 3.2033994}
For Cluster 4, the average state distances are: {0: 4.5766826, 1: 3.2233033}
For Cluster 5, the average state distances are: {0: 4.5842586, 1: 3.2173722}
